# Data Cleaning and Compiling

In this project, I use more than a half dozen data sources for my final model. Here, you can see how I manipulated and cleaned the data in order create the most accurate model I could.

In [1]:
import pandas as pd
import numpy as np

To begin, I imported data from the Pennsylvania Secretary of State on election results from 2000-2018, and compiled them with voter registration data from them. The below function cleans and aggregates these datasets.

In [2]:
def cleaning_dataframe(results, registration, year):
    results.columns = ['election_name', 'COUNTY', 'office_name', 'district_name',
       'party_name', 'candidate_name', 'votes', "yes_votes", "no_votes"]
    results.drop(columns=['election_name', 'yes_votes', 'no_votes'], inplace=True)
    results.drop_duplicates(inplace=True)
    results['district_name'] = [str(word).lower() for word in results['district_name']]
    results['elected_office'] = np.where(results['district_name'] == 'statewide',
                                     results['office_name'],
                                     results['district_name'])
    results['votes'] = results['votes'].str.replace(",","").astype(int)
    final = pd.DataFrame()
    final['COUNTY'] = results['COUNTY'].unique()
    final["id"] = [f'{county}_{year}' for county in final['COUNTY']]
    final['year'] = year
    votes_by_county = pd.pivot_table(results, values=['votes'],
                                 index = ['elected_office', 'COUNTY'],
                                 aggfunc='sum').reset_index()
    max_votes = votes_by_county.groupby('COUNTY').max().reset_index()
    final = final.merge(max_votes, on='COUNTY', how='outer')
    final.rename(columns = {'votes':'turnout'}, inplace=True)    
    final['elections'] = [results[results['COUNTY'] == county]['elected_office'].unique().shape[0] for county in final['COUNTY']]
    final['candidates'] = [results[results['COUNTY'] == county]['candidate_name'].unique().shape[0] for county in final['COUNTY']]
    temp = results.groupby(['COUNTY', 'elected_office']).count()
    uncontested_totals = temp[temp['office_name'] == 1].groupby('COUNTY').count().reset_index()[['COUNTY', 'votes']]
    final = final.merge(uncontested_totals, on='COUNTY', how='outer')
    final.rename(columns = {'votes':'uncontested_elections'}, inplace=True)
    final.fillna(0, inplace=True)
    final.replace('McKEAN', 'MCKEAN', inplace=True)
    registration.columns = ['COUNTY', 'DEM', 'REP', 'ALL PARTIES']
    registration.replace('LACKWANNA', 'LACKAWANNA', inplace=True)
    final = final.merge(registration, on='COUNTY')
    
    return final

In [3]:
#2018
results_2018 = pd.read_csv('../2018_results.CSV')
RV_2018 = pd.read_csv('../2018_VR.csv')

#2016
results_2016 = pd.read_csv('../2016_results.CSV')
RV_2016 = pd.read_csv('../2016_VR.csv')

#2014
results_2014 = pd.read_csv('../2014_results.CSV')
RV_2014 = pd.read_csv('../2014_VR.csv')

# #2012
results_2012 = pd.read_csv('../2012_results.CSV')
RV_2012 = pd.read_csv('../2012_VR.csv')

#2010
results_2010 = pd.read_csv('../2010_results.CSV')
RV_2010 = pd.read_csv('../2010_VR.csv')

#2008
results_2008 = pd.read_csv('../2008_results.CSV')
RV_2008 = pd.read_csv('../2008_VR.csv')

#2006
results_2006 = pd.read_csv('../2006_results.CSV')
RV_2006 = pd.read_csv('../2006_VR.csv')

#2004
results_2004 = pd.read_csv('../2004_results.CSV')
RV_2004 = pd.read_csv('../2004_VR.csv')

#2002
results_2002 = pd.read_csv('../2002_results.CSV')
RV_2002 = pd.read_csv('../2002_VR.csv')

#2000
results_2000 = pd.read_csv('../2000_results.CSV')
RV_2000 = pd.read_csv('../2000_VR.csv')

In [5]:
final_2018 = cleaning_dataframe(results_2018, RV_2018, 2018)
final_2016 = cleaning_dataframe(results_2016, RV_2016, 2016)
final_2014 = cleaning_dataframe(results_2014, RV_2014, 2014)
final_2012 = cleaning_dataframe(results_2012, RV_2012, 2012)
final_2010 = cleaning_dataframe(results_2010, RV_2010, 2010)
final_2008 = cleaning_dataframe(results_2008, RV_2008, 2008)
final_2006 = cleaning_dataframe(results_2006, RV_2006, 2006)
final_2004 = cleaning_dataframe(results_2004, RV_2004, 2004)
final_2002 = cleaning_dataframe(results_2002, RV_2002, 2002)
final_2000 = cleaning_dataframe(results_2000, RV_2000, 2000)

In [6]:
#Appended new turnout data from 2000 due to data inaccuracy spotted during modeling process.
#No other years appreared to have problems.

In [7]:
turnout_add = pd.read_csv('../tunrout_appendage.csv')

In [8]:
final_2000['turnout'] = turnout_add[turnout_add['Year'] == 2000]['Turnout']

In [9]:
final_rv_and_results = pd.concat([final_2018, final_2016, final_2014, final_2012, final_2010, final_2000, final_2008, final_2006, final_2004, final_2002])

In [10]:
final_rv_and_results.reset_index(inplace=True)

In [54]:
final_2018.head()

,COUNTY,id,year,elected_office,turnout,elections,candidates,uncontested_elections,DEM,REP,ALL PARTIES
0,ADAMS,ADAMS_2018,2018,United States Senator,39102,5,14,0.0,19557,36652,67025
1,ALLEGHENY,ALLEGHENY_2018,2018,United States Senator,541703,29,47,15.0,546641,261938,941028
2,ARMSTRONG,ARMSTRONG_2018,2018,United States Senator,24643,6,14,2.0,14419,22211,41350
3,BEAVER,BEAVER_2018,2018,United States Senator,67929,8,19,1.0,55569,41149,110681
4,BEDFORD,BEDFORD_2018,2018,United States Senator,18863,5,14,0.0,7906,20587,31487


### Add in demographic data from various sources

Much of this data needed to be cleaned and modified in order to be usable by a model.

In [12]:
#import employment data from BLS
employment = pd.read_csv('../employment_data.csv',  names=['COUNTY', 'YEAR', 'per_unemployed'])

In [13]:
employment['COUNTY'] = [name.split()[0].upper() for name in employment['COUNTY']]
employment['YEAR_2'] = [str(year) for year in employment['YEAR']]
employment['id'] = employment['COUNTY']+'_'+employment['YEAR_2']

In [14]:
append_unemployment = employment.drop(columns=['YEAR_2', 'COUNTY', 'YEAR'])

In [15]:
econ_data = pd.read_csv('../Book8.csv')

In [16]:
econ_data['COUNTY'] = [name.split()[0].upper() for name in econ_data['COUNTY']]
econ_data['YEAR_2'] = [str(year) for year in econ_data['YEAR']]
econ_data['id'] = employment['COUNTY']+'_'+econ_data['YEAR_2']
econ_append = econ_data.drop(columns=['YEAR_2', 'COUNTY', 'YEAR'])

#### Import Census Data

In [17]:
#import census data
census_pt_1 = pd.read_csv('../PA_census_data_2010_2017.csv')
census_pt_2 = pd.read_csv('../PA_census_data_2000_2010.csv')

In [18]:
census_pt_1['YEAR'] = census_pt_1['YEAR'].map({3:2010, 4:2011, 5:2012,
                                               6:2013, 7:2014, 8:2015,
                                               9:2016, 10:2018})

census_pt_2['YEAR'] = census_pt_2['YEAR'].map({2:2000, 3:2001, 4:2002,
                                               5:2003, 6:2004, 7:2005,
                                               8:2006, 9:2007, 10:2008,
                                               11:2009})
census_pt_1['COUNTY'] = [name.split()[0].upper() for name in census_pt_1['CTYNAME']]
census_pt_2['COUNTY'] = [name.split()[0].upper() for name in census_pt_2['CTYNAME']]
census_pt_1.dropna(inplace=True)
census_pt_2.dropna(inplace=True)

In [19]:
columns = ['COUNTY', 'YEAR', 'AGEGRP', 'TOT_POP', 'TOT_MALE',
             'TOT_FEMALE', 'NHWA_MALE', 'NHWA_FEMALE','BA_MALE',
             'BA_FEMALE', 'H_MALE','H_FEMALE', 'AA_MALE', 'AA_FEMALE',
             'TOM_MALE', 'TOM_FEMALE']

In [20]:
census_data = pd.concat([census_pt_1, census_pt_2])
census_data = census_data[columns].reset_index()
#total male, total female, 

/anaconda3/envs/DSI/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [21]:
census_data['YEAR'] = [int(year) for year in census_data['YEAR']]
census_data['YEAR_2'] = [str(year) for year in census_data['YEAR']]
census_data['id'] = census_data['COUNTY']+'_'+census_data['YEAR_2']

In [23]:
aggregate_census = census_data[(census_data['AGEGRP'] == 99) | ((census_data['AGEGRP'] == 0) & (census_data['YEAR'] >= 2010))] 

In [58]:
aggregate_census.sort_values(by = 'percent_female')

,index,COUNTY,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,NHWA_MALE,NHWA_FEMALE,BA_MALE,...,TOM_MALE,TOM_FEMALE,YEAR_2,id,percent_female,percent_white,percent_black,percent_hispanic,percent_asian,percent_mixed_race
4085,5111,FOREST,2018,0,7297,4999,2298,3069,2196,1502,...,35,34,2018,FOREST_2018,0.314924,0.721529,0.207620,0.064136,0.001782,0.009456
4066,5092,FOREST,2016,0,7306,4958,2348,3082,2253,1462,...,35,32,2016,FOREST_2016,0.321380,0.730222,0.201752,0.062414,0.001779,0.009171
4047,5073,FOREST,2015,0,7357,4976,2381,3101,2285,1460,...,36,32,2015,FOREST_2015,0.323637,0.732092,0.200082,0.061982,0.001903,0.009243
3990,5016,FOREST,2012,0,7587,5094,2493,3256,2404,1433,...,38,30,2012,FOREST_2012,0.328588,0.746013,0.190457,0.057862,0.002109,0.008963
3971,4997,FOREST,2011,0,7671,5147,2524,3315,2443,1438,...,36,28,2011,FOREST_2011,0.329031,0.750619,0.188893,0.056055,0.001564,0.008343
4028,5054,FOREST,2014,0,7445,4988,2457,3153,2355,1426,...,37,34,2014,FOREST_2014,0.330020,0.739825,0.193284,0.060578,0.002015,0.009537
3952,4978,FOREST,2010,0,7711,5158,2553,3374,2473,1405,...,34,28,2010,FOREST_2010,0.331085,0.758267,0.183374,0.054338,0.001556,0.008040
4009,5035,FOREST,2013,0,7534,5039,2495,3228,2395,1412,...,38,31,2013,FOREST_2013,0.331165,0.746350,0.189143,0.058933,0.001726,0.009158
15583,6979,FOREST,2009,99,7737,5152,2585,3356,2506,1414,...,34,29,2009,FOREST_2009,0.334109,0.757658,0.183792,0.054285,0.001551,0.008143
15543,6939,FOREST,2007,99,7512,4961,2551,3335,2475,1280,...,29,28,2007,FOREST_2007,0.339590,0.773429,0.171326,0.050985,0.001597,0.007588


In [24]:
aggregate_census['percent_female'] = aggregate_census['TOT_FEMALE']/aggregate_census['TOT_POP']
aggregate_census['percent_white'] = (aggregate_census['NHWA_MALE'] + aggregate_census['NHWA_FEMALE'])/aggregate_census['TOT_POP']
aggregate_census['percent_black'] = (aggregate_census['BA_MALE'] + aggregate_census['BA_FEMALE'])/aggregate_census['TOT_POP']
aggregate_census['percent_hispanic'] = (aggregate_census['H_MALE'] + aggregate_census['H_FEMALE'])/aggregate_census['TOT_POP']
aggregate_census['percent_asian'] = (aggregate_census['AA_MALE'] + aggregate_census['AA_FEMALE'])/aggregate_census['TOT_POP']
aggregate_census['percent_mixed_race'] = (aggregate_census['TOM_MALE'] + aggregate_census['TOM_FEMALE'])/aggregate_census['TOT_POP']

/anaconda3/envs/DSI/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/DSI/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/DSI/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [25]:
census_append = aggregate_census[['id', 'percent_female', 'percent_white', 'percent_hispanic', 'percent_asian',
                                  'percent_mixed_race', 'percent_black', 'TOT_POP']]

In [26]:
census_append['id'].unique().shape

(1206,)

In [27]:
#aggregate age data
census_data['AGEGRP'] = census_data['AGEGRP'].map({4:'15-34', 5:'15-34', 6:'15-34',
                                                   7:'15-34', 
                                                   8:'35-54', 9:'35-54', 10:'35-54',
                                                   11:'35-54', 12:'55+', 13:'55+',
                                                   14:'55+', 15:'55+', 16:'55+',
                                                   17:'55+', 18:'55+'
                                                  })

In [28]:
age_groups = census_data.dropna()

In [29]:
age_groups = pd.pivot_table(age_groups, index=['COUNTY', 'AGEGRP', 'id'], values =['TOT_POP'], aggfunc='sum').reset_index()

In [30]:
age_groups = pd.pivot_table(age_groups, index=['id'], values = ['TOT_POP'], columns = ['AGEGRP']).reset_index()

In [31]:
all_columns = final_rv_and_results.merge(census_append, on='id')

In [32]:
with_ages = all_columns.merge(age_groups, on='id')

/anaconda3/envs/DSI/lib/python3.7/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/anaconda3/envs/DSI/lib/python3.7/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [33]:
with_ages.columns = ['index',                'COUNTY',
                          'id',                  'year',
              'elected_office',               'turnout',
                   'elections',            'candidates',
       'uncontested_elections',                   'DEM',
                         'REP',           'ALL PARTIES',
              'percent_female',         'percent_white',
            'percent_hispanic',         'percent_asian',
          'percent_mixed_race',         'percent_black',
                     'TOT_POP',    '15-34',
          '35-54',      '55+']

In [34]:
with_ages.head()

with_ages['percent_25_34'] = with_ages['15-34']/with_ages['TOT_POP']
with_ages['percent_55_older'] = with_ages['55+']/with_ages['TOT_POP']
with_ages['percent_35_54'] = with_ages['35-54']/with_ages['TOT_POP']

In [35]:
appending = with_ages.merge(append_unemployment, on='id')

#### Import Educational Data

In [36]:
ed_levels = pd.read_csv('../ed_levels.csv')

In [37]:
ed_levels.columns

Index(['county', 'ed rate', 'year', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')

In [38]:
ed_levels['county'] = [name.split(',')[0].upper() for name in ed_levels['county']]
ed_levels['YEAR_2'] = [str(year) for year in ed_levels['year']]
ed_levels['id'] = ed_levels['county']+'_'+ed_levels['YEAR_2']
ed_append = ed_levels[['id', 'ed rate']]

### Join All Data together and modify variables for best results

In [39]:
appending = appending.merge(ed_append, on='id')

In [40]:
final = appending.merge(econ_append, on='id')

In [41]:
final = final.drop(columns=['15-34', '35-54', '55+'])

In [42]:
final['turnout_percent'] = final['turnout']/final['ALL PARTIES']

In [43]:
final['dem_percent'] = final['DEM']/final['ALL PARTIES']

In [44]:
final['rep_percent'] = final['REP']/final['ALL PARTIES']

#### Import data about prior tunrout and append

In [45]:
prior_turnout = pd.read_csv('../four_years_prior_turnout.csv')

In [46]:

prior_turnout['YEAR_2'] = [str(year) for year in prior_turnout['year']]
prior_turnout['id'] = prior_turnout['COUNTY']+'_'+prior_turnout['YEAR_2']
prior_turnout = prior_turnout[['id', 'turnout_4_years_prior']]

In [47]:
final = final.merge(prior_turnout, on='id')

#### Map national political climate data onto appropriate years

In [48]:
final['presidential'] = final['year'].map({2018:0, 2016:1, 2014:0,
                                               2012:1, 2010:0, 2008:1,
                                               2006:0, 2004:1, 2002:0, 2000:1})
final['R_pres'] = final['year'].map({2018:1, 2016:0, 2014:0,
                                               2012:0, 2010:0, 2008:1,
                                               2006:1, 2004:1, 2002:1, 2000:0})
final['incumbent_top_of_ticket'] = final['year'].map({2018:1, 2016:0, 2014:1,
                                               2012:1, 2010:0, 2008:1,
                                               2006:1, 2004:1, 2002:0, 2000:0})
#turnout data from https://www.idea.int/data-tools/country-view/295/40
final['national_turnout'] = final['year'].map({2018:.493, 2016:.6544, 2014:.4250,
                                               2012:.6444, 2010:.4859, 2008:.6436,
                                               2006:.4752, 2004:.6875, 2002:.4531, 2000:.6376})

In [49]:
final['percent_uncontested'] = final['uncontested_elections']/final['elections']
final['percent_registered'] = final['TOT_POP']/final['ALL PARTIES']
final['how_partisan'] = np.abs(final['dem_percent']-final['rep_percent'])

In [50]:
final.columns

Index(['index', 'COUNTY', 'id', 'year', 'elected_office', 'turnout',
       'elections', 'candidates', 'uncontested_elections', 'DEM', 'REP',
       'ALL PARTIES', 'percent_female', 'percent_white', 'percent_hispanic',
       'percent_asian', 'percent_mixed_race', 'percent_black', 'TOT_POP',
       'percent_25_34', 'percent_55_older', 'percent_35_54', 'per_unemployed',
       'ed rate', 'per_poverty', 'med_income', 'turnout_percent',
       'dem_percent', 'rep_percent', 'turnout_4_years_prior', 'presidential',
       'R_pres', 'incumbent_top_of_ticket', 'national_turnout',
       'percent_uncontested', 'percent_registered', 'how_partisan'],
      dtype='object')

## Export Data for Modeling

In [51]:
final.to_csv('../may_15_update.csv')